In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import json
import pandas as pd
import numpy as np
import requests
import time
import qdrant_client
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from sklearn.metrics import ndcg_score
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse

In [6]:
# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [7]:
QDRANT_URL = "http://192.68.10.50:6333"
client = QdrantClient(url=QDRANT_URL)

In [8]:
queries = {q["_id"]: q["text"] for q in map(json.loads, open("queries.jsonl"))}
corpus = {c["_id"]: c["text"] for c in map(json.loads, open("SciFact.jsonl"))} # corpus.jsonl
qrels = pd.read_csv("train.tsv", sep="\t", names=["query-id", "corpus-id", "score"]) #qrels/train.tsv

In [26]:
print(qrels["query-id"].unique()[:10])

['query-id' '0' '2' '4' '6' '9' '10' '11' '12' '14']


In [9]:
# Select 10 queries for evaluation
selected_queries = qrels["query-id"].unique()[:10]

In [10]:
def retrieve_top_k(query_text, k=10):
    """Retrieve top-k results from Qdrant for a given query."""
    query_vector = model.encode(query_text).tolist()
    results = client.search(collection_name="scifact", query_vector=query_vector, limit=k)
    return [r.payload["id"] for r in results]  # Retrieve document IDs

In [11]:
def compute_metrics(retrieved_docs, relevant_docs):
    """Compute DCG, NDCG, Recall@10, and MAP@10."""
    relevance_scores = [1 if doc in relevant_docs else 0 for doc in retrieved_docs]
    
    dcg = sum(rel / np.log2(i+2) for i, rel in enumerate(relevance_scores))
    ideal_dcg = sum(1 / np.log2(i+2) for i in range(len(relevant_docs)))
    ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
    
    recall_at_10 = sum(relevance_scores) / len(relevant_docs)
    
    precisions = [sum(relevance_scores[:i+1]) / (i+1) for i in range(len(relevance_scores)) if relevance_scores[i] > 0]
    map_at_10 = np.mean(precisions) if precisions else 0
    
    return dcg, ndcg, recall_at_10, map_at_10

In [12]:
# Run multiple evaluations to check determinism
iterations = 3  # Can be increased to 5 later
results = []

In [13]:
for query_id in selected_queries:
    query_text = queries.get(query_id, "")
    relevant_docs = set(qrels[qrels["query-id"] == query_id]["corpus-id"])

In [ ]:
def safe_retrieve_top_k(query_text, k=10, max_retries=3):
    """Wrapper for retrieve_top_k with error handling"""
    for attempt in range(max_retries):
        try:
            return retrieve_top_k(query_text, k)
        except (requests.exceptions.ConnectionError, UnexpectedResponse) as e:
            print(f"Attempt {attempt + 1}/{max_retries} failed: {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(1)  # Wait before retry

# Modified query loop
for query_id in selected_queries:
    query_text = queries.get(query_id, "")
    relevant_docs = set(qrels[qrels["query-id"] == query_id]["corpus-id"])
    
    row = {"Query ID": query_id}
    for i in range(iterations):
        try:
            retrieved_docs = safe_retrieve_top_k(query_text, k=10)
            dcg, ndcg, recall, map_ = compute_metrics(retrieved_docs, relevant_docs)
            row[f"Iteration {i+1}"] = f"NDCG: {ndcg:.3f}, Recall@10: {recall:.3f}, MAP@10: {map_:.3f}"
        except Exception as e:
            print(f"Error processing query {query_id}, iteration {i+1}: {e}")
            row[f"Iteration {i+1}"] = "Error: Failed to retrieve results"
    
    results.append(row)

C:\Users\BeloAbhigyan\AppData\Local\Temp\ipykernel_22332\3409790786.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name="scifact", query_vector=query_vector, limit=k)


In [21]:
print(client.scroll(collection_name="scifact", limit=1))

([Record(id=0, payload={'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the

In [20]:
# 1. Test server availability first
import requests
import time
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse

def test_qdrant_connection():
    try:
        response = requests.get("http://192.168.10.50:6333/collections")
        print(f"Server connection test: {response.status_code}")
        return True
    except:
        return False

# 2. Initialize client with better settings
if test_qdrant_connection():
    client = QdrantClient(
        url="http://192.168.10.50:6333",
        timeout=60.0,  # Increased timeout
        prefer_grpc=False
    )
    
    # 3. Try scrolling with error handling
    try:
        result = client.scroll(
            collection_name="scifact",
            limit=1,
            timeout=30
        )
        print(result)
    except Exception as e:
        print(f"Error during scroll: {e}")
        print("Try using IP 127.0.0.1 instead of localhost")
else:
    print("Could not connect to Qdrant server")

Server connection test: 200
([Record(id=0, payload={'id': '4983', 'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.', 'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. 

In [22]:
test_query = "What is the effect of vaccines?"
retrieved_docs = retrieve_top_k(test_query, k=5)
print(retrieved_docs)


['8063697', '33934971', '27527854', '8069939', '25897733']


C:\Users\BeloAbhigyan\AppData\Local\Temp\ipykernel_22332\3409790786.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name="scifact", query_vector=query_vector, limit=k)


In [24]:
df = pd.DataFrame(results)
df.to_excel("evaluation_results.xlsx", index=False)
print("Evaluation completed. Results saved to evaluation_results.xlsx")

Evaluation completed. Results saved to evaluation_results.xlsx


In [27]:
# df.head()

,Query ID,Iteration 1,Iteration 2,Iteration 3
0,query-id,Error: Failed to retrieve results,Error: Failed to retrieve results,Error: Failed to retrieve results
1,0,Error: Failed to retrieve results,Error: Failed to retrieve results,Error: Failed to retrieve results
2,2,Error: Failed to retrieve results,Error: Failed to retrieve results,Error: Failed to retrieve results
3,4,Error: Failed to retrieve results,Error: Failed to retrieve results,Error: Failed to retrieve results
4,6,Error: Failed to retrieve results,Error: Failed to retrieve results,Error: Failed to retrieve results


In [28]:
# print("DataFrame Info:")
# print(df.info())

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Query ID     16 non-null     object
 1   Iteration 1  16 non-null     object
 2   Iteration 2  16 non-null     object
 3   Iteration 3  16 non-null     object
dtypes: object(4)
memory usage: 644.0+ bytes
None


In [29]:
print("DataFrame Info:")
print(df.info())

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Query ID     16 non-null     object
 1   Iteration 1  16 non-null     object
 2   Iteration 2  16 non-null     object
 3   Iteration 3  16 non-null     object
dtypes: object(4)
memory usage: 644.0+ bytes
None


In [30]:
print("\nDuplicate counts:")
print(df['Query ID'].value_counts())


Duplicate counts:
Query ID
query-id    3
0           2
2           2
4           2
6           2
9           1
10          1
11          1
12          1
14          1
Name: count, dtype: int64


In [31]:
print("\nSample Rows:")
print(df.head())


Sample Rows:
   Query ID                        Iteration 1  \
0  query-id  Error: Failed to retrieve results   
1         0  Error: Failed to retrieve results   
2         2  Error: Failed to retrieve results   
3         4  Error: Failed to retrieve results   
4         6  Error: Failed to retrieve results   

                         Iteration 2                        Iteration 3  
0  Error: Failed to retrieve results  Error: Failed to retrieve results  
1  Error: Failed to retrieve results  Error: Failed to retrieve results  
2  Error: Failed to retrieve results  Error: Failed to retrieve results  
3  Error: Failed to retrieve results  Error: Failed to retrieve results  
4  Error: Failed to retrieve results  Error: Failed to retrieve results  
